In [ ]:
import torch
import numpy as np
from torch import nn 
import random
from torch.nn import functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
imiona = open("imionapl.txt", encoding = "UTF-8").read().splitlines()
imiona = [i.lower() for i in imiona]
imiona[:5]

['ada', 'adalbert', 'adam', 'adela', 'adelajda']

In [ ]:
chars = sorted(list(set(''.join(imiona))))
stoi = {s:i +1  for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i :s for s, i in stoi.items()}
vocab_size = len(itos)
print(vocab_size)

31


In [ ]:
block_size = 3 
def build_dataset(imiona):  
    X, Y = [], []
    for w in imiona:

        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y
random.seed(42)
random.shuffle(imiona)
n1 = int(0.8*len(imiona))
n2 = int(0.9*len(imiona))

X_train, Y_train = build_dataset(imiona[:n1])
X_dev, Y_dev = build_dataset(imiona[n1:n2])
X_test, Y_test = build_dataset(imiona[n2:])
    

torch.Size([3919, 3]) torch.Size([3919])
torch.Size([510, 3]) torch.Size([510])
torch.Size([480, 3]) torch.Size([480])


In [177]:
dims = 2 
hidden_neurons = 10

g = torch.Generator().manual_seed(42)
C = torch.randn((vocab_size, dims))
W1 = torch.randn((dims*block_size, hidden_neurons), generator = g)
b1 = torch.randn((hidden_neurons), generator = g)
W2 = torch.randn((hidden_neurons, hidden_neurons), generator = g)
W3 = torch.randn((hidden_neurons, vocab_size), generator = g)

parameters = [C, W1, W2,W3, b1]

def enable_grad(parameters):
    for p in parameters:
        p.requires_grad = True
enable_grad(parameters)

In [197]:

def model(X_train,Y_train, batch_size, parameters, max_steps, state = None):
    outputs = []  
    C, W1, W2, W3,  b1 = parameters
    for i in range(max_steps):

        # minibatch construct
        ix = torch.randint(0, X_train.shape[0], (batch_size,))
        X_batched, Y_batched = X_train[ix], Y_train[ix]
        # forward pass
        emb = C[X_batched]
        #print(f"""emb :{emb.shape}""")
        embcat = emb.view(emb.shape[0], -1)
        if state is None:
            # Initial state with shape: (batch_size, hidden_neurons)
            state = torch.zeros((embcat.shape[0], hidden_neurons))
        else:
            state = state
         
        state = torch.tanh(embcat @ W1 + state @ W2 + b1)
        outputs.append(state)
        logits = state @ W3 
        loss = F.cross_entropy(logits, Y_batched)
        cat = torch.argmax(logits, dim=1)
        accuracy = (cat == Y_batched).float().mean()
        print(loss)
    return outputs, state
   
        



In [198]:

bs = 8
hidden_neurons = hidden_neurons
out, states = model(X_train, Y_train, bs , parameters, hidden_neurons)

tensor(6.3464, grad_fn=<NllLossBackward0>)
tensor(5.7045, grad_fn=<NllLossBackward0>)
tensor(6.4246, grad_fn=<NllLossBackward0>)
tensor(8.3777, grad_fn=<NllLossBackward0>)
tensor(6.8384, grad_fn=<NllLossBackward0>)
tensor(7.0815, grad_fn=<NllLossBackward0>)
tensor(6.2059, grad_fn=<NllLossBackward0>)
tensor(6.5466, grad_fn=<NllLossBackward0>)
tensor(5.2808, grad_fn=<NllLossBackward0>)
tensor(5.1683, grad_fn=<NllLossBackward0>)


In [199]:
def check_len(a, n):  
    """Check the length of a list."""
    print(len(a))
    assert len(a) == n, f'list\'s length {len(a)} != expected length {n}'

def check_shape(a, shape): 
    """Check the shape of a tensor."""
    assert a.shape == shape, \
            f'tensor\'s shape {a.shape} != expected shape {shape}'

In [200]:
check_len(out, 10)
check_shape(out[0], (bs, hidden_neurons))
check_shape(states, (bs, hidden_neurons))

10
